In [ ]:
import os
new_directory = '/home/fruitspec-lab/FruitSpec/Code/roi/fsCounter'
os.chdir(new_directory)
current_directory = os.getcwd()
print("Current working directory:", current_directory)

In [ ]:
from MHS.F_model_training import *
from vision.misc.help_func import go_up_n_levels
from vision.pipelines.fe_pipeline import *
from vision.feature_extractor.vegetation_indexes import num_deno_nan_divide, num_deno_nan_divide_np, ndvi_cuda

# os.chdir(r'/home/fruitspec-lab/FruitSpec/Code/roi/fsCounter/MHS')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from vision.tools.image_stitching import plot_2_imgs
from vision.tools.camera import is_saturated
from vision.feature_extractor.stat_tools import get_mode
from vision.depth.slicer.slicer import find_tree_height_limits
from vision.misc.help_func import go_up_n_levels, validate_output_path


In [ ]:
def preprocess(frame, xyz_frame, resize_facotr, show_imgs, vndvi_filter, vari_filter, channels_thresh):
    org_frame = frame
    if resize_facotr > 1:
        new_shape = (frame.shape[1] // resize_facotr, frame.shape[0] // resize_facotr)
        frame = cv2.resize(frame, new_shape)
        xyz_frame = cv2.resize(xyz_frame, new_shape)
    else:
        frame, xyz_frame = np.copy(frame), np.copy(xyz_frame)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    flatten_frame = gray_frame.flatten()
    gray_mode = get_mode(flatten_frame)
    if show_imgs:
        plt.hist(flatten_frame, density = True, bins = 50)
        plt.title(gray_mode)
        plt.show()
    if gray_mode < 25: # Dark image
        vndvi_filter = vndvi_filter - 0.01
        if np.sum(vari_filter) > 0:
            vari_filter = (0.5, 100)
        channels_thresh[2] = (0, 255)
        channels_thresh[1] = (0, 255)
    return org_frame, frame, xyz_frame, vndvi_filter, vari_filter, channels_thresh

def apply_depth_filter(frame, xyz_frame, depth_filter, show_imgs):
    if np.sum(depth_filter) > 0:
        if show_imgs:
            frame_pre = frame.copy()
        depth_chanel = xyz_frame[:,:,2]
        frame[depth_chanel < depth_filter[0]] = 0
        frame[depth_chanel > depth_filter[1]] = 0
        if show_imgs:
            plot_2_imgs(frame_pre, frame, title = "pre depth filter,    post filter")
    return frame

def channel_filter(frame, channels_thresh, show_imgs):
    if show_imgs:
        frame_pre = frame.copy()
    mask = np.ones_like(frame[:,:,0], dtype=bool)
    for channel, thresholds in channels_thresh.items():
        if not np.sum(thresholds):
            continue
        chan = frame[:, :, channel]
        channel_mask = cv2.inRange(chan, thresholds[0], thresholds[1])
        mask = np.bitwise_and(mask, channel_mask)
    frame[np.where(mask==0)] = 0
    if show_imgs:
        plot_2_imgs(frame_pre, frame, title="prefilter channel, post filter")
    return frame

def gmr_filter(gree_minus_red_thres, frame, show_imgs):
    if np.sum(gree_minus_red_thres) != 0:
        diff = frame[:, :, 1] - frame[:, :, 0]
        valids = cv2.inRange(diff, gree_minus_red_thres[0], gree_minus_red_thres[1])
        if show_imgs:
            frame_pre = frame.copy()
        frame[np.where(valids==0)] = 0
        if show_imgs:
            plot_2_imgs(frame_pre, frame, title = "prefilter g minus r,    post filter")
    return frame

def gmb_filter(gree_minus_blue_thres, frame, show_imgs):
    if np.sum(gree_minus_blue_thres) !=0:
        diff = (frame[:, :, 1] - frame[:, :, 2])/(frame[:, :, 1]+1)
        valids = cv2.inRange(diff, gree_minus_blue_thres[0], gree_minus_blue_thres[1])
        if show_imgs:
            frame_pre = frame.copy()
        frame[np.where(valids==0)] = 0
        if show_imgs:
            plot_2_imgs(frame_pre, frame, title = "prefilter g minus b,    post filter")
    return frame

def apply_vari_filter(frame, Red, Green, Blue, vari_filter, show_imgs):
    vari = num_deno_nan_divide_np((Green - Red), (Green + Red - Blue))
    vari[~np.isfinite(vari)] = 0
    vari = np.clip(vari, -5,15)
    if show_imgs:
        frame_pre = frame.copy()
    frame_copy = frame.copy()
    frame_copy[vari >= vari_filter[1]]= 0
    frame_copy[vari <= vari_filter[0]]= 0
    frame[-int(frame.shape[0]/4):,:] = frame_copy[-int(frame.shape[0]/4):,:]
    if show_imgs:
        plot_2_imgs(frame_pre, frame, title = "prefilter vari,    post filter")
    return frame

def apply_vndvi_filter(frame, Red, Green, Blue, vndvi_filter, show_imgs):
    vndvi = 0.5268 * (Red**(-0.1294) * Green**(0.3389)* Blue**(-0.3118))
    vndvi[~np.isfinite(vndvi)] = np.nanmin(vndvi)
    if show_imgs:
        frame_pre = frame.copy()
    frame[vndvi <= vndvi_filter] = np.nanmin(vndvi)
    if show_imgs:
        plot_2_imgs(frame_pre, frame, title = "prefilter vndvi,    post filter")
    return frame

def show_vi_filter_res(org_frame, vari_filter, vndvi_filter):
    Red, Green, Blue = org_frame[:,:,0], org_frame[:,:,1], org_frame[:,:,2]
    vari = num_deno_nan_divide_np((Green - Red), (Green + Red - Blue))
    vari[~np.isfinite(vari)] = 0
    vari = np.clip(vari, -5,10)   
    vndvi = 0.5268 * (Red**(-0.1294) * Green**(0.3389)* Blue**(-0.3118))
    vndvi[~np.isfinite(vndvi)] = np.nanmin(vndvi)
    print(np.nanquantile(vndvi, 0.025), np.nanquantile(vndvi, 0.975))
    plt.hist(vndvi.flatten(), bins = 50)
    plt.show()
    plot_2_imgs(vari, np.clip(vndvi, np.nanquantile(vndvi, 0.025), np.nanquantile(vndvi, 0.975)), title = "vari,    ndvi")
    plot_2_imgs(cv2.inRange(vari, vari_filter[0], vari_filter[1]), vndvi > vndvi_filter, title = "vari filtered,    ndvi filtred")
    
def debug_func(frame, Red, Green, Blue):
#     vndvi = 0.5268 * (Red**(-0.1294) * Green**(0.3389)* Blue**(-0.3118))
#     vari = num_deno_nan_divide_np((Green - Red), (Green + Red - Blue))
#     vari[~np.isfinite(vari)] = np.nanmin(vari)
#     vari = np.clip(vari, -5,10)
#     plt.hist(vari.flatten(), bins = 50)
#     plt.show()    
    diff0 = frame[:, :, 1] - frame[:, :, 0]
    diff2 = frame[:, :, 1] - frame[:, :, 2]
    plot_2_imgs(diff0, diff2, title = "diff_0, diff_2")
    diff0 = (frame[:, :, 1] - frame[:, :, 0])/(frame[:, :, 1]+1)
    diff2 = (frame[:, :, 1] - frame[:, :, 2])/(frame[:, :, 1]+1)
    plot_2_imgs(diff0, diff2, title = "diff_0/c1, diff_2/c1")
    diff0 = (frame[:, :, 1] - frame[:, :, 0])/(frame[:, :, 1]+frame[:, :, 0]+1)
    diff2 = (frame[:, :, 1] - frame[:, :, 2])/(frame[:, :, 1]+frame[:, :, 2]+1)
    plot_2_imgs(diff0, diff2, title = "diff_0/sum, diff_2/sum")    
#     vndvi[~np.isfinite(vndvi)] = np.nanmin(vndvi)
#     plot_2_imgs(vari, vndvi, title = "Vari, vNDVI")
#     plot_2_imgs(vari < 0, vari < 0.5, title = "Vari, vNDVI")
#     plot_2_imgs(vari < 1, vari < 1.5, title = "Vari, vNDVI")
#     plot_2_imgs(vari < 2, vari < 2.5, title = "Vari, vNDVI")
#     plot_2_imgs(vari < 3, vari < 3.5, title = "Vari, vNDVI")
#     plot_2_imgs(vari < 4, vari < 4.5, title = "Vari, vNDVI")
#     plot_2_imgs(vari <= 5, vari < 5.5, title = "Vari, vNDVI")

In [ ]:
def calculate_gradients(xyz_frame, mid_height, include_y_grad):
    depth_channel = xyz_frame[mid_height:, :, 2]
    depth_grad = np.gradient(depth_channel)[0]

    y_grad = None
    if include_y_grad:
        y_channel = xyz_frame[mid_height:, :, 1]
        y_grad = np.gradient(y_channel)[0]

    return depth_grad, y_grad


def apply_thresholding(depth_grad, y_grad, min_grad = 0.01):
    depth_grad_mask = depth_grad > min_grad

    y_grad_mask = None
    if y_grad is not None:
        y_grad_mask = y_grad > min_grad

    return depth_grad_mask, y_grad_mask


def combine_masks(depth_grad_mask, y_grad_mask):
    if y_grad_mask is not None:
        return depth_grad_mask.astype(np.uint8) | y_grad_mask.astype(np.uint8)
    else:
        return depth_grad_mask.astype(np.uint8)


def dilate_image(comb_img, img_h):
    comb_img = cv2.erode(comb_img, np.ones((img_h//200+1, img_h//200 +1)), iterations=1)
    return cv2.dilate(comb_img, np.ones((img_h//40+1, img_h//40 +1)), iterations=1)


def find_filtered_contours(comb_img_dilated, min_contour_area):
    contours, _ = cv2.findContours(comb_img_dilated.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours, [contour for contour in contours if cv2.contourArea(contour) >= min_contour_area*10]


def create_output_image(xyz_frame, contours_filtered_drawed, mid_height):
    height, width = contours_filtered_drawed.shape
    output_img = np.zeros_like(xyz_frame)

    for x in range(width):
        column = contours_filtered_drawed[:, x]
        min_y = len(column) - np.argmax(column[::-1]) - 1 + mid_height
        output_img[min_y:, x] = 1

    return output_img


def segment_floor(xyz_frame, show_imgs=True, min_contour_area=100, include_y_grad=False):
    img_h = xyz_frame.shape[0]
    mid_height = img_h // 2

    depth_grad, y_grad = calculate_gradients(xyz_frame, mid_height, include_y_grad)
    depth_grad_mask, y_grad_mask = apply_thresholding(depth_grad, y_grad)

    if show_imgs and y_grad_mask is not None:
        plot_2_imgs(depth_grad_mask, y_grad_mask, title="masked grads (depth, y)")

    comb_img = combine_masks(depth_grad_mask, y_grad_mask)
    comb_img_dilated = dilate_image(comb_img, img_h)

    if show_imgs:
        plot_2_imgs(comb_img, comb_img_dilated, title="org gradient image, dilated image")

    contours, filtered_contours = find_filtered_contours(comb_img_dilated, min_contour_area)
    contours_filtered_drawed = cv2.drawContours(comb_img_dilated.copy(), filtered_contours, -1, (255, 0, 0), 1)

    if show_imgs:
        contours_drawed = cv2.drawContours(comb_img_dilated.copy(), contours, -1, (255, 0, 0), 1)
        plot_2_imgs(contours_drawed, contours_filtered_drawed, title="contours, filtered contours")

    output_img = create_output_image(xyz_frame, contours_filtered_drawed, mid_height)

    if show_imgs:
        plot_2_imgs(output_img, contours_filtered_drawed, title="final image, filtered contours")

    return output_img

def remove_floor(frame, xyz_frame, show_imgs=True):
    if show_imgs:
        frame_pre = frame.copy()
    frame[segment_floor(xyz_frame, show_imgs)==1] = 0
    if show_imgs:
        plot_2_imgs(frame_pre, frame, title = "prefilter floor,    post floor filter")
    return frame
        

In [ ]:
def thresh_channels(frame, xyz_frame, channels_thresh={0: (0, 175), 1: (10, 255), 2: (0, 200)},
                    gree_minus_red_thres=(0, 0), gree_minus_blue_thres=(0, 0), vari_filter = (0, 0), vndvi_filter = 0.35,
                    depth_filter = (0.75, 5),show_imgs=False, resize_facotr = 4, full_tree_thresh = 0.33):
    """
    Thresholds the input frame based on the given channel thresholds and green minus red thresholds.

    Args:
        frame (ndarray): The input frame to threshold.
        channels_thresh (dict): A dictionary containing the channel indices as keys and a tuple of thresholds as values.
        gree_minus_red_thres (tuple): A tuple containing the green minus red thresholds.

    Returns:
        ndarray: The thresholded frame.

    """
    org_frame, frame, xyz_frame, vndvi_filter, vari_filter, channels_thresh = preprocess(frame, xyz_frame, 
                                                                              resize_facotr, show_imgs,
                                                                              vndvi_filter, vari_filter, 
                                                                              channels_thresh)
    frame = remove_floor(frame, xyz_frame) #, show_imgs)
    frame = apply_depth_filter(frame, xyz_frame, depth_filter, show_imgs)
    if np.sum(vari_filter) + vndvi_filter > 0:
        Red, Green, Blue = frame[:,:,0], frame[:,:,1], frame[:,:,2]
#     if show_imgs:
#     debug_func(frame, Red, Green, Blue)
    
    frame = channel_filter(frame, channels_thresh, show_imgs)
    frame = gmr_filter(gree_minus_red_thres, frame, show_imgs)
    frame = gmb_filter(gree_minus_blue_thres, frame, show_imgs)
    if np.sum(vari_filter) > 0:
        frame = apply_vari_filter(frame, Red, Green, Blue, vari_filter, show_imgs)
    if vndvi_filter > 0:
        frame = apply_vndvi_filter(frame, Red, Green, Blue, vndvi_filter, show_imgs)
    if show_imgs:
        show_vi_filter_res(org_frame, vari_filter, vndvi_filter)
    threshed_img = cv2.threshold(frame[:, :, 1], 0, 255, cv2.THRESH_BINARY)[1] > 0
    full_tree = np.mean(threshed_img[:threshed_img.shape[0]//15,:]) < full_tree_thresh
    return threshed_img, full_tree

def get_percent_h_seen(zed_threshed_cut, y1, y2):
    return round(np.mean(np.nansum(zed_threshed_cut[y1:y2, :], axis = 0) / np.nansum(zed_threshed_cut, axis = 0)),2 )

def get_percent_seen(zed_frame, xyz_frame, coors, return_threshed=False, show_imgs=False, resize_facotr=4):
    x1, y1, x2, y2 = [cor//resize_facotr for cor in coors]
    zed_threshed, full_tree = thresh_channels(zed_frame, xyz_frame,
                                                  show_imgs=show_imgs, resize_facotr=resize_facotr)
    zed_threshed_cut = zed_threshed[:, x1:x2]
    percent_seen = round(np.nansum(zed_threshed_cut[y1:y2, :])/np.nansum(zed_threshed_cut),2)
    percent_h_seen = get_percent_h_seen(zed_threshed_cut, y1, y2)
    percent_seen_top = round(np.nansum(zed_threshed_cut[y1:y2, :])/np.nansum(zed_threshed_cut[:y2,:]),2)
    no_tree_indicator = np.mean(zed_threshed) < 0.2
    if return_threshed:
        return percent_seen, percent_h_seen, percent_seen_top, no_tree_indicator, full_tree, zed_threshed_cut
    return percent_seen, percent_h_seen, percent_seen_top, no_tree_indicator, full_tree

def debug_percent_seen_batch(batch_fsi, batch_rgb_zed, batch_zed, frame_ids, b_align, debug_folder=""):
    for jai, zed, zed_xyz, f_id, cors in zip(batch_fsi, batch_rgb_zed, batch_zed, frame_ids, b_align):
        cors = [int(cor) for cor in cors[:4]]
        x1, y1, x2, y2 = cors
        zed_copy = zed.copy()
        color = (0, 0, 255)
        thickness = 2
        percent_seen, percent_h_seen, percent_seen_top, no_tree_indicator, full_tree, zed_threshed = get_percent_seen(zed_copy,
                                                                                                          zed_xyz,
                                                                                           cors[:4], True, False)
        cv2.rectangle(zed_copy, (x1, y1), (x2, y2), color, thickness)
        plot_name = os.path.join(debug_folder, f"jai_zed_{f_id}.jpg")
        plot_name_2 = os.path.join(debug_folder, f"threshed_zed_{f_id}.jpg")
        plot_2_imgs(jai, zed_copy, title=f_id, save_to=plot_name, save_only=False, cv2_save=False, quick_save=False)
        plot_2_imgs(zed_threshed, zed_copy, title=f"""frame: {f_id}, percent: {percent_seen}, percent_top: {percent_seen_top}, percent_h_seen: {percent_h_seen}
        full_tree: {full_tree}, no_tree_indicator: {no_tree_indicator}""",
                    save_to=plot_name_2, save_only=False, cv2_save=False, quick_save=False)


In [ ]:
def run_on_row_ps(row_scan_path, min_slice_len, direction, tree_id, frames, shift, debug_folder=""):
    adts_loader = None
    validate_jai_zed_json(row_scan_path)
    customer_name = os.path.basename(go_up_n_levels(row_scan_path, 4))
    block_name = os.path.basename(go_up_n_levels(row_scan_path, 3))
    row_name = "R" + os.path.basename(go_up_n_levels(row_scan_path, 1)).split("_")[-1]
    if debug_folder != "":
        debug_folder = os.path.join(debug_folder, f"{block_name}_{row_name}")
    validate_output_path(debug_folder)
    validate_slice_data(row_scan_path, min_slice_len=min_slice_len, direction=direction)
    metadata, _ = get_metadata(row_scan_path)
    fe, adts_loader, batch_size = init_fe_obj(row_scan_path, tree_id, adts_loader, False, direction)
    n_batchs = len(frames) // batch_size
    for i in tqdm(range(n_batchs)):
        frame_ids = frames[i * batch_size: (i + 1) * batch_size]
        batch_res = adts_loader.load_batch(frame_ids, shift)
        batch_fsi, batch_zed, batch_jai_rgb, batch_rgb_zed, batch_tracker, batch_slicer, frame_ids, b_align, b_jai_translation, bps = batch_res
        debug_percent_seen_batch(batch_fsi, batch_rgb_zed, batch_zed, frame_ids, b_align, debug_folder)
        

In [ ]:
min_slice_len=5
direction="left"
frames = [str(num) for num in range(100,110)]
tree_id = 0
shift = 0
os.chdir(new_directory)
debug_folder = "/media/fruitspec-lab/easystore/ps_debug"

In [ ]:
os.chdir(new_directory)
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/DEWAGD/DWDBLE33/190123/row_23/2"
frames = [str(num) for num in range(955,963)]
run_on_row_ps(row_scan_path, min_slice_len, direction, tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/MOTCHA/BEERAMU0/220823/row_107/1"
frames = [str(num) for num in range(56,69)]
run_on_row_ps(row_scan_path, min_slice_len, direction, tree_id, frames, shift, debug_folder)

In [ ]:
os.chdir(new_directory)
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/MOTCHA/BEERAMU0/220823/row_107/1"
frames = [str(num) for num in range(100,109)]
run_on_row_ps(row_scan_path, min_slice_len, direction, tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/MOTCHA/ORSCHIST/230813/row_6/1"
frames = [str(num) for num in range(403,415)]
run_on_row_ps(row_scan_path, min_slice_len, direction, tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/MOTCHA/ORSCHIST/230813/row_5/1"
frames = [str(num) for num in range(298,310)]
run_on_row_ps(row_scan_path, min_slice_len, direction, tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/MOTCHA/RAUSTENB/060723/row_2/1"
frames = [str(num) for num in range(223,227)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/MOTCHA/RAUSTENB/060723/row_3/1"
frames = [str(num) for num in range(562,566)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/MOTCHA/MEIRAVHA/230723/row_101/1"
frames = [str(num) for num in range(193,197)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path ="/media/fruitspec-lab/cam175/customers_new/PROPAL/M7XXXXXX/250323/row_23/1"
frames = [str(num) for num in range(1000,1009)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/PROPAL/FUKUMOTO/260323/row_22/1"
frames = [str(num) for num in range(2000, 2009)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/PROPAL/2018XXXX/230323/row_62/1"
frames = [str(num) for num in range(1500, 1509)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/MOTCHA/MICHAL22/240723/row_101/1"
frames = [str(num) for num in range(60,68)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/MOTCHA/MICHAL22/240723/row_102/2"
frames = [str(num) for num in range(200,209)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/FOWLER/BLOCK700/170723/row_1/1"
frames = [str(num) for num in range(200,209)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/FOWLER/BLOCK700/170723/row_4/1"
frames = [str(num) for num in range(416,428)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/FOWLER/BLAYNEY0/170723/row_10/1"
frames = [str(num) for num in range(336,344)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/FOWLER/BLAYNEY0/170723/row_12/1"
frames = [str(num) for num in range(380,388)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/FOWLER/BLOCKAT0/140823/row_4/1"
frames = [str(num) for num in range(745,757)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/FOWLER/BLOCKAT0/140823/row_7/1"
frames = [str(num) for num in range(3296,3312)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/FOWLER/BLOCKDX0/row_9/1"
frames = [str(num) for num in range(1360,1368)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/FOWLER/BLOCKDX0/row_10/1"
frames = [str(num) for num in range(370,378)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/FOWLER/FREDIANI/170723/row_14/1"
frames = [str(num) for num in range(200,208)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/FOWLER/FREDIANI/170723/row_15/1"
frames = [str(num) for num in range(397,409)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/FOWLER/G60CLEM0/140823/row_5/1"
frames = [str(num) for num in range(3199,3215)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/FOWLER/G60CLEM0/140823/row_11/1"
frames = [str(num) for num in range(663,671)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/FOWLER/MAZMANI2/170723/row_20/1"
frames = [str(num) for num in range(506, 510)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/FOWLER/MAZMANI2/170723/row_21/1"
frames = [str(num) for num in range(362,366)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/FOWLER/MAZMANIA/170723/row_17/1"
frames = [str(num) for num in range(316,324)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/FOWLER/MAZMANIA/170723/row_18/1"
frames = [str(num) for num in range(268,276)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/FOWLER/OLIVER55/170723/row_6/1"
frames = [str(num) for num in range(66,82)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)

In [ ]:
row_scan_path = "/media/fruitspec-lab/cam175/customers_new/FOWLER/OLIVER55/170723/row_6/1"
frames = [str(num) for num in range(603,615)]
run_on_row_ps(row_scan_path, min_slice_len, "right", tree_id, frames, shift, debug_folder)